<a href="https://colab.research.google.com/github/msm13/agents-course/blob/main/code_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Alfred is planning a party at the Wayne family mansion and needs your help to ensure everything goes smoothly. To assist him, we'll apply what we've learned about how a multi-step `CodeAgent` operates.


## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
!pip install smolagents -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.7
    Uninstalling huggingface_hub-1.3.7:
      Successfully uninstalled huggingface_hub-1.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Selecting a Playlist for the Party Using `smolagents`

An important part of a successful party is the music. Alfred needs some help selecting the playlist. Luckily, `smolagents` has got us covered! We can build an agent capable of searching the web. To give the agent access to this tool, we include it in the tool list when creating the agent.

For the model, we'll rely on `InferenceClientModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Running an agent is quite straightforward:

In [3]:
from smolagents import CodeAgent, WebSearchTool, InferenceClientModel

agent = CodeAgent(tools=[WebSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="best music recommendations for a party at Wayne's mansion")                   
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
Creating the perfect soundtrack is crucial to getting the right atmosphere at your event. If you're throwing a 
party , there are countless songs that will get everyone up and dancing; even once you've chosen your mix, you'll 
probably still think of more later! So, to keep things simple, we've compiled the best  party songs of all time, 
from timeless classics to contemporary hits, giving ...

[late nights in the wayne manor - YouTube Music](https://music.youtube.com/watch?v=oOdHCjj3jn4)
these are songs that bruce wayne listens to while wandering the halls of the great wayne manor. find this playlist 
on my spotify: https://open.spotify.com/u...

[Best Songs To Party: DJ's Ultimate Party Songs Playlist](https://top40weekly.com/best-songs-to-party/)
Planning a celebration? Don't forget the music ! We've compiled the best songs to party that are sure to get 
everyone in the mood to dance and have a good time.

[The Ultimate Playlist to Get the Party Started](https://alekasgettogether.com/top-songs-for-any-party/)
Discover the ultimate party playlist for 2025 with over 50 top songs to keep your guests dancing all night long.

[The 10 Best Party Music Playlist Ideas for Any Vibe in 2025](https://1021events.com/best-party-music-playlist/)
Craft the ultimate soundtrack with our guide to the 10 best  party  music playlist ideas, from 80s throwbacks to 
current hits. Perfect for any event!

[200 Classic House Party Songs Everyone Knows | The Best ... - 
iSpyTunes](https://www.ispytunes.com/post/house-party-songs)
Ready to throw the ultimate house party ? These 200 songs will have everyone on their feet! From classic jams to 
modern hits, we got you.

[GUIDE TO Planning Music for a Party - 
partiesuniverse.com](https://partiesuniverse.com/guide-to-planning-music-for-a-party/)
There are relatively few high-quality guides about how to plan brilliant music  for  a  party , so we have decided 
to create our own. Our how-to guide for planning the best  music  for  a  party , is exactly what you need to throw
an epic celebration where the music completely carries the vibe.

[WAYNE soundtrack - playlist by Charlie Linskey | 
Spotify](https://open.spotify.com/playlist/5iVbp78KnovfXzpwSOgzDB)
 WAYNE soundtrack · Playlist · 46 songs · 8.4K likes Preview of Spotify Sign up to get unlimited songs and podcasts
with occasional ads. No credit card needed.

[10 Best Music Playlists for Parties to Ignite Your Night 
(2026)](https://www.playlistnames.org/best-music-playlists-for-parties/)
Throwing a party and want to be that host—the one who keeps the dance floor packed and the good vibes flowing? 
You're in the right place. We've curated the 10 best  music playlists for parties that cover every vibe, from 
high-energy EDM bangers to chill background beats perfect for cocktail hour. Did you know that 72% of party -goers 
say bad music is the top reason they leave early? Yikes ...

[Looking for party music ideas? Our top 10 party 
playlists](https://www.mgnevents.co.uk/insights/party-music-ideas-top-10-playlists/)
Along with lighting & decoration, music really impacts the atmosphere of a party . Check out our top 10 party 
playlists to set the mood

Out: None

[Step 1: Duration 11.65 seconds| Input tokens: 2,093 | Output tokens: 902]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The best music recommendations for a party at Wayne's mansion include the 'WAYNE soundtrack'       
  Spotify playlist, 'The 75 Best Party Songs That Will Get Everyone Dancing' from Gear4music, and 'DJ's Ultimate   
  Party Songs Playlist' from top40weekly.com for high-energy dance tracks.")                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The best music recommendations for a party at Wayne's mansion include the 'WAYNE soundtrack' Spotify 
playlist, 'The 75 Best Party Songs That Will Get Everyone Dancing' from Gear4music, and 'DJ's Ultimate Party Songs 
Playlist' from top40weekly.com for high-energy dance tracks.

[Step 2: Duration 21.82 seconds| Input tokens: 5,160 | Output tokens: 5,319]

"The best music recommendations for a party at Wayne's mansion include the 'WAYNE soundtrack' Spotify playlist, 'The 75 Best Party Songs That Will Get Everyone Dancing' from Gear4music, and 'DJ's Ultimate Party Songs Playlist' from top40weekly.com for high-energy dance tracks."

When you run this example, the output will **display a trace of the workflow steps being executed**. It will also print the corresponding Python code with the message:

```python
 ─ Executing parsed code: ────────────────────────────────────────────────────────────────────────────────────────
  results = web_search(query="best music for a Batman party")                                                      
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
```

After a few steps, you'll see the generated playlist that Alfred can use for the party! 🎵

## Using a Custom Tool to Prepare the Menu

Now that we have selected a playlist, we need to organize the menu for the guests. Again, Alfred can take advantage of `smolagents` to do so. Here, we use the `@tool` decorator to define a custom function that acts as a tool. We'll cover tool creation in more detail later, so for now, we can simply run the code.

As you can see in the example below, we will create a tool using `@tool` decorator and include it in the `tools` list.

In [4]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = suggest_menu(occasion="formal")                                                                           
  final_answer(menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 3-course dinner with wine and dessert.

[Step 1: Duration 3.73 seconds| Input tokens: 2,112 | Output tokens: 576]

'3-course dinner with wine and dessert.'

The agent will run for a few steps until finding the answer.

The menu is ready! 🥗

## Using Python Imports Inside the Agent

We have the playlist and menu ready, but we need to check one more crucial detail: preparation time!

Alfred needs to calculate when everything would be ready if he started preparing now, in case they need assistance from other superheroes.

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

**Code execution has strict security measures** - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in `additional_authorized_imports`.
For more details on secure code execution, see the official [guide](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution).

When creating the agent, we will use `additional_authorized_imports` to allow for importing the `datetime` module.

In [5]:
from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=InferenceClientModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
  current_time = datetime.datetime.now()                                                                           
  end_time = current_time + datetime.timedelta(hours=3)                                                            
  formatted_time = end_time.strftime("%H:%M")                                                                      
  final_answer(formatted_time)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 02:57

[Step 1: Duration 15.81 seconds| Input tokens: 2,111 | Output tokens: 3,260]

'02:57'

These examples are just the beginning of what you can do with code agents, and we're already starting to see their utility for preparing the party.
You can learn more about how to build code agents in the [smolagents documentation](https://huggingface.co/docs/smolagents).

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

## Sharing Our Custom Party Preparator Agent to the Hub

Wouldn't it be **amazing to share our very own Alfred agent with the community**? By doing so, anyone can easily download and use the agent directly from the Hub, bringing the ultimate party planner of Gotham to their fingertips! Let's make it happen! 🎉

The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:


In [6]:
from smolagents import CodeAgent, WebSearchTool, InferenceClientModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        WebSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=InferenceClientModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to find a playlist suitable for a villain masquerade party at Wayne's mansion. Since Wayne's       
mansion is associated with Batman, the theme likely involves Batman villains. I'll start by searching for "Batman  
villain masquerade party playlist" to get relevant song suggestions.                                               
                                                                                                                   
<code>                                                                                                             
search_results = web_search(query="Batman villain masquerade party playlist")                                      
print(search_results)                                                                                              
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="Batman villain masquerade party playlist")                                    
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Batman villain arc - playlist by PurplePumpkin | 
Spotify](https://open.spotify.com/playlist/7itheXFSylncgZPl14IAXm)
 Batman  villain arc · Playlist · 300 songs · 58 likes

[Batman: The Ultimate Collection Playlist | WaterTower 
Music](https://www.youtube.com/playlist?list=PLBKadB95sF47mJ-Nx787RE3q5Vgzl6Xgj)
Happy 85th birthday to Batman ! Celebrate with this action-packed playlist honoring the one and only Batman . 
Batman : The Ultimate Collection Playlist | WaterT...

[Stream RhymeRider | Listen to Batman's Villains playlist online for 
...](https://soundcloud.com/rhymerider/sets/batmans-villains)
Listen to Batman's  Villains , a playlist curated by RhymeRider on desktop and mobile.

[Playing from - YouTube Music](https://music.youtube.com/watch?v=CH8a01P_OJY)
Playing from Batman  Villains  Playlist Vol. 1 - The Long Halloween Mix Save Autoplay Add similar content to the 
end of the queue

[you're at a masquerade and unknowingly dancing with the 
villain](https://open.spotify.com/playlist/4zxYvQs9gByjRhAn5ujSfQ)
you're at a masquerade and unknowingly dancing with the villain · Playlist · 89 songs · 10 likes

[Batman Music Party Mix](https://www.youtube.com/playlist?list=PLONmfYACg6Z7IdC_2LAGKMY69VB2q_Gfb)
 Batman Music Party Mix by Thierry Orvoen • Playlist • 2,142 videos • No views Play all Unavailable videos are 
hidden 1

[Smash Pages Playlist: Happy Batman Day](https://smashpages.net/2021/09/18/smash-pages-playlist-happy-batman-day/)
Break out your cape and cowl and get ready to do the Batusi — it's Batman Day, as declared by the fine folks at DC 
and Warner Bros., so we're back with another Smash Pages playlist featuring songs about the Dark Knight, his 
partners and even his villains . So maybe skip going to the movies with your parents today and instead stay in and 
jam to songs about Bill Finger and Bob Kane's most ...

[Stream BATMAN ️ | Listen to villain arc playlist ... - 
SoundCloud](https://soundcloud.com/william-wendorf/sets/batman-arc-1)
Listen to villain arc, a playlist curated by BATMAN ️🦇 on desktop and mobile.

[Masquerade party music - playlist by drew - Spotify](https://open.spotify.com/playlist/3gICKSayCeN6muUWNGHeK4)
 Playlist · Masquerade  party music 🎭 · 38 items · 31 saves

[masquerade party - playlist by ben - Spotify](https://open.spotify.com/playlist/4Gz0k5xGJ13Lb4CT4JEBK7)
 Playlist · masquerade  party 🎭 · 176 items · 37 saves

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
final_answer("you're at a masquerade and unknowingly dancing with the villain")                                    
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("you're at a masquerade and unknowingly dancing with the villain")                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: you're at a masquerade and unknowingly dancing with the villain

[Step 2: Duration 15.29 seconds| Input tokens: 5,488 | Output tokens: 4,736]

"you're at a masquerade and unknowingly dancing with the villain"

In [7]:
agent.push_to_hub('mmichiels13/AlfredAgent')

HfHubHTTPError: (Request ID: Root=1-69868015-7a08ddd30a10ca8773ea7ff6;540e4c5e-5245-4de3-953c-3da6300920df)

403 Forbidden: You don't have the rights to create a space under the namespace "mmichiels13".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

To download the agent again, use the code below:

In [ ]:
agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub('mmichiels13/AlfredAgent', trust_remote_code=True)

In [ ]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

What's also exciting is that shared agents are directly available as Hugging Face Spaces, allowing you to interact with them in real-time. You can explore other agents [here](https://huggingface.co/spaces/davidberenstein1957/smolagents-and-tools).

For example, the _AlfredAgent_ is available [here](https://huggingface.co/spaces/sergiopaniego/AlfredAgent).

### Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

Full trace can be found [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z).

As Alfred fine-tunes the Party Preparator Agent, he's growing weary of debugging its runs. Agents, by nature, are unpredictable and difficult to inspect. But since he aims to build the ultimate Party Preparator Agent and deploy it in production, he needs robust traceability for future monitoring and analysis.  

Once again, `smolagents` comes to the rescue! It embraces the [OpenTelemetry](https://opentelemetry.io/) standard for instrumenting agent runs, allowing seamless inspection and logging. With the help of [Langfuse](https://langfuse.com/) and the `SmolagentsInstrumentor`, Alfred can easily track and analyze his agent’s behavior.  

Setting it up is straightforward!  

First, we need to install the necessary dependencies:  

In [ ]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

Next, Alfred has already created an account on Langfuse and has his API keys ready. If you haven’t done so yet, you can sign up for Langfuse Cloud [here](https://cloud.langfuse.com/) or explore [alternatives](https://huggingface.co/docs/smolagents/tutorials/inspect_runs).  

Once you have your API keys, they need to be properly configured as follows:

In [ ]:
import os
import base64
from google.colab import userdata

LANGFUSE_PUBLIC_KEY=userdata.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=userdata.get("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

Finally, Alfred is ready to initialize the `SmolagentsInstrumentor` and start tracking his agent's performance.  

In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Alfred is now connected 🔌! The runs from `smolagents` are being logged in Langfuse, giving him full visibility into the agent's behavior. With this setup, he's ready to revisit previous runs and refine his Party Preparator Agent even further.  

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub('mmichiels13/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

Alfred can now access this logs [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z) to review and analyze them.  

Meanwhile, the [suggested playlist](https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw) sets the perfect vibe for the party preparations. Cool, right? 🎶
